# idact - Prometheus sandbox

## Initial setup

Add `idact` to path:

In [2]:
import sys
import os
import bitmath
import getpass
import contextlib
import fabric
import logging
import subprocess
from pprint import pprint

def append_idact_path():
    idact_path = os.path.realpath(os.path.join(os.getcwd(), '../'))
    sys.path.append(idact_path)
append_idact_path()

from idact import *
from idact.detail.auth.set_password import set_password

os.environ['IDACT_KEY_LOCATION'] = os.path.join(os.getcwd(), '../.notebook-ssh')
# os.environ['IDACT_KEY_LOCATION'] = os.path.expanduser('~/.ssh')
if not os.path.exists(os.environ['IDACT_KEY_LOCATION']):
    os.mkdir(os.environ['IDACT_KEY_LOCATION'])

USER = 'plggarstka'

Hide debug information, setup context manager stack (for testing purposes)

## Add cluster (only first run)

In [3]:
key = KeyType.RSA  # Generate RSA key
# key = os.path.join(os.path.expanduser('~/.ssh'), 'id_rsa')

In [4]:
cluster = add_cluster(name="pro",
                      user=USER,
                      host="pro.cyfronet.pl",
                      port=22,
                      auth=AuthMethod.PUBLIC_KEY,
                      key=key,
                      install_key=True,
                      scratch="$SCRATCH")
save_environment('.idact-env')

2018-10-22 02:25:54 INFO: Generating public-private key pair.


## Load cluster (subsequent runs)

In [5]:
load_environment('.idact-env')
cluster = show_cluster("pro")
cluster

Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='E:\\shared\\uni\\eng-project\\notebooks\\../.notebook-ssh\\id_rsa_sy', install_key=True, disable_sshd=False)

In [6]:
set_log_level(logging.INFO)
#set_log_level(logging.DEBUG)
save_environment('.idact-env')

In [7]:
node = cluster.get_access_node()
node

Node(pro.cyfronet.pl:22, None)

On your first action, you will be asked for a password to install the key.
You can connect explicitly (optional) to do this right now:

In [8]:
node.connect()

2018-10-22 02:26:15 INFO: Installing key using password authentication.
Password for plggarstka@pro.cyfronet.pl:22: ········


In [9]:
node.run('whoami')

'plggarstka'

In [10]:
node.run('hostname')

'login01.pro.cyfronet.pl'

## Allocate nodes

In [11]:
nodes = cluster.allocate_nodes(nodes=2,
                               cores=2,
                               memory_per_node=bitmath.GiB(10),
                               walltime=Walltime(minutes=20),
                               native_args={
                                   '--partition': 'plgrid-testing',
                                   '--account': 'intdata'
                               })

2018-10-22 02:26:42 INFO: Creating the ssh directory.


In [12]:
nodes

Nodes([Node(NotAllocated),Node(NotAllocated)], SlurmAllocation(job_id=13791710))

In [13]:
nodes.wait()
nodes

Nodes([Node(p0118:35295, 2018-10-22 00:46:56.709952+00:00),Node(p0235:52865, 2018-10-22 00:46:56.709952+00:00)], SlurmAllocation(job_id=13791710))

## Run commands

In [14]:
nodes[0].run('whoami')

'plggarstka'

In [15]:
nodes[0].run('hostname')

'p0118'

In [16]:
nodes[1].run('squeue')

'JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)\n          13791710 plgrid-te     wrap plggarst  R       0:18      2 p[0118,0235]'

In [17]:
nodes[1].run('hostname')

'p0235'

## Examine node resources

In [18]:
nodes[0].resources.memory_total

GiB(10.0)

In [19]:
nodes[0].resources.memory_usage

GiB(0.022457122802734375)

In [20]:
nodes[0].resources.cpu_cores

2

In [21]:
nodes[0].resources.cpu_usage

1.0

## Tunnel

In [22]:
tunnel = nodes[0].tunnel(here=9000, there=10000)

2018-10-22 02:27:20,195| ERROR   | Could not establish connection from ('127.0.0.1', 51576) to remote side of the tunnel
2018-10-22 02:27:20,197| ERROR   | Exception: Error reading SSH protocol banner
2018-10-22 02:27:20,201| ERROR   | Traceback (most recent call last):
2018-10-22 02:27:20,202| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\transport.py", line 2044, in _check_banner
2018-10-22 02:27:20,202| ERROR   |     buf = self.packetizer.readline(timeout)
2018-10-22 02:27:20,203| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\packet.py", line 353, in readline
2018-10-22 02:27:20,203| ERROR   |     buf += self._read_timeout(timeout)
2018-10-22 02:27:20,204| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\packet.py", line 542, in _read_timeout
2018-10-22 02:27:20,204| ERROR   |     raise EOFError()
2018-10-22 02:27:20,205| ERROR   | EOFError
2018-10-22 02:27:20,205| ERROR   | 
2018-10-22 02:27:20,206| ERROR

2018-10-22 02:27:20 WARNING: Exception: Could not establish session to SSH gateway, retry 1/3.


In [23]:
tunnel

MultiHopTunnel(9000:10000)

In [24]:
tunnel.close()

## Deploy notebook

One-time config step (cluster-specific):

In [25]:
cluster.config.setup_actions.jupyter = ['module load plgrid/tools/python-intel/3.6.2']
save_environment('.idact-env')

To run Jupyter Notebook on the cluster:

In [26]:
nb = nodes[0].deploy_notebook(local_port=8080)
nb

2018-10-22 02:27:50,666| ERROR   | Could not establish connection from ('127.0.0.1', 51586) to remote side of the tunnel
2018-10-22 02:27:50,668| ERROR   | Exception: Error reading SSH protocol banner
2018-10-22 02:27:50,670| ERROR   | Traceback (most recent call last):
2018-10-22 02:27:50,670| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\transport.py", line 2044, in _check_banner
2018-10-22 02:27:50,671| ERROR   |     buf = self.packetizer.readline(timeout)
2018-10-22 02:27:50,671| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\packet.py", line 353, in readline
2018-10-22 02:27:50,671| ERROR   |     buf += self._read_timeout(timeout)
2018-10-22 02:27:50,672| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\packet.py", line 542, in _read_timeout
2018-10-22 02:27:50,672| ERROR   |     raise EOFError()
2018-10-22 02:27:50,672| ERROR   | EOFError
2018-10-22 02:27:50,673| ERROR   | 
2018-10-22 02:27:50,673| ERROR

2018-10-22 02:27:50 WARNING: Exception: Could not establish session to SSH gateway, retry 1/3.


JupyterDeployment(8080 -> Node(p0118:35295, 2018-10-22 00:46:56.709952+00:00)

In [27]:
nodes[0].resources.memory_usage

GiB(0.08470535278320312)

In [28]:
nb.local_port

8080

To open the deployed notebook server in a new tab:

In [29]:
nb.open_in_browser()

In [30]:
nodes[0].resources.memory_usage

GiB(0.08750152587890625)

### Push and pull notebook

You can access the deployed notebook from multiple places by first pushing it:

In [31]:
cluster.push_deployment(nb)

2018-10-22 02:28:29 INFO: Pushing deployment: JupyterDeployment(8080 -> Node(p0118:35295, 2018-10-22 00:46:56.709952+00:00)


And then pulling:

In [32]:
deployments = cluster.pull_deployments()
deployments.jupyter_deployments

2018-10-22 02:28:44 INFO: Pulling deployments.


2018-10-22 02:29:19,009| ERROR   | Could not open connection to gateway


2018-10-22 02:29:19 WARNING: Exception: Could not establish session to SSH gateway, retry 1/3.


2018-10-22 02:29:26,226| ERROR   | Could not establish connection from ('127.0.0.1', 51616) to remote side of the tunnel
2018-10-22 02:29:26,228| ERROR   | Exception: Error reading SSH protocol banner
2018-10-22 02:29:26,230| ERROR   | Traceback (most recent call last):
2018-10-22 02:29:26,231| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\transport.py", line 2044, in _check_banner
2018-10-22 02:29:26,231| ERROR   |     buf = self.packetizer.readline(timeout)
2018-10-22 02:29:26,231| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\packet.py", line 353, in readline
2018-10-22 02:29:26,232| ERROR   |     buf += self._read_timeout(timeout)
2018-10-22 02:29:26,232| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\packet.py", line 542, in _read_timeout
2018-10-22 02:29:26,233| ERROR   |     raise EOFError()
2018-10-22 02:29:26,233| ERROR   | EOFError
2018-10-22 02:29:26,234| ERROR   | 
2018-10-22 02:29:26,234| ERROR

2018-10-22 02:29:26 WARNING: Exception: Could not establish session to SSH gateway, retry 1/3.
2018-10-22 02:29:31 INFO: Pulled Jupyter deployment: JupyterDeployment(51625 -> Node(p0118:35295, 2018-10-22 00:46:56.709952+00:00)


[JupyterDeployment(51625 -> Node(p0118:35295, 2018-10-22 00:46:56.709952+00:00)]

In [33]:
nb_2 = deployments.jupyter_deployments[0]
nb_2

JupyterDeployment(51625 -> Node(p0118:35295, 2018-10-22 00:46:56.709952+00:00)

In [34]:
nb_2.open_in_browser()

In [35]:
nb_2.cancel()

2018-10-22 02:29:46 INFO: Cancelling Jupyter deployment.


More on pushing and pulling deployments in the next section.

## Push and pull nodes

In order to work with Dask, you would usually need a notebook running on the cluster, as shown above.

To access the allocated nodes from the cluster, you need to push their deployment first, same as the notebook deployment above:

In [37]:
cluster.push_deployment(nodes)

2018-10-22 02:32:05 INFO: Pushing deployment: Nodes([Node(p0118:35295, 2018-10-22 00:46:56.709952+00:00),Node(p0235:52865, 2018-10-22 00:46:56.709952+00:00)], SlurmAllocation(job_id=13791710))


Then, you would pull the deployment on the cluster:

In [38]:
deployments = cluster.pull_deployments()
deployments

2018-10-22 02:32:31 INFO: Pulling deployments.


2018-10-22 02:32:40,286| ERROR   | Could not establish connection from ('127.0.0.1', 51651) to remote side of the tunnel
2018-10-22 02:32:40,288| ERROR   | Exception: Error reading SSH protocol banner
2018-10-22 02:32:40,290| ERROR   | Traceback (most recent call last):
2018-10-22 02:32:40,291| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\transport.py", line 2044, in _check_banner
2018-10-22 02:32:40,291| ERROR   |     buf = self.packetizer.readline(timeout)
2018-10-22 02:32:40,292| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\packet.py", line 353, in readline
2018-10-22 02:32:40,292| ERROR   |     buf += self._read_timeout(timeout)
2018-10-22 02:32:40,292| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\packet.py", line 542, in _read_timeout
2018-10-22 02:32:40,293| ERROR   |     raise EOFError()
2018-10-22 02:32:40,293| ERROR   | EOFError
2018-10-22 02:32:40,293| ERROR   | 
2018-10-22 02:32:40,294| ERROR

2018-10-22 02:32:40 WARNING: Exception: Could not establish session to SSH gateway, retry 1/3.
2018-10-22 02:32:45 INFO: Pulled allocation deployment: Nodes([Node(p0118:35295, 2018-10-22 00:46:56.709952+00:00),Node(p0235:52865, 2018-10-22 00:46:56.709952+00:00)], SlurmAllocation(job_id=13791710))
2018-10-22 02:32:45 INFO: Pulled Jupyter deployment: JupyterDeployment(51660 -> Node(p0118:35295, 2018-10-22 00:46:56.709952+00:00)


2018-10-22 02:32:48,664| ERROR   | Secsh channel 0 open FAILED: Connection refused: Connect failed
2018-10-22 02:32:48,667| ERROR   | Could not establish connection from ('127.0.0.1', 51660) to remote side of the tunnel


2018-10-22 02:32:48 WARNING: Exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',)), retry 1/3.


...

2018-10-22 02:32:52,924| ERROR   | Secsh channel 2 open FAILED: Connection refused: Connect failed
2018-10-22 02:32:52,925| ERROR   | Could not establish connection from ('127.0.0.1', 51660) to remote side of the tunnel


2018-10-22 02:32:52 WARNING: Exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',)), retry 3/3.


2018-10-22 02:32:54,961| ERROR   | Secsh channel 3 open FAILED: Connection refused: Connect failed
2018-10-22 02:32:54,962| ERROR   | Could not establish connection from ('127.0.0.1', 51660) to remote side of the tunnel


2018-10-22 02:32:54 WARNING: Discarding a Jupyter deployment,  because it is no longer functional: JupyterDeployment(51660 -> Node(p0118:35295, 2018-10-22 00:46:56.709952+00:00). Exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',)).


SynchronizedDeployments(nodes=1, jupyter_deployments=0)

In [39]:
nodes = deployments.nodes[0]
nodes

Nodes([Node(p0118:35295, 2018-10-22 00:46:56.709952+00:00),Node(p0235:52865, 2018-10-22 00:46:56.709952+00:00)], SlurmAllocation(job_id=13791710))

Essentially, this feature is intended for using an allocation in multiple notebooks at once.

Deployments are cleared automatically if they are expired or cancelled. They can also be cleared manually by  running:

In [40]:
cluster.clear_pushed_deployments()

2018-10-22 02:33:08 INFO: Clearing deployments.


## idact-notebook app

You can deploy nodes and notebook automatically using the following command:
```
idact-notebook
```
or its equivalent, if you didn't install idact using pip:
```
python -m idact.notebook
```
Help message:

In [41]:
help_message = subprocess.getoutput(
    "cd .. && {python} -m idact.notebook --help".format(
        python=sys.executable))
print(help_message)

Usage: notebook.py [OPTIONS] CLUSTER_NAME

  A console script that executes a Jupyter Notebook instance on an allocated
  cluster node, and makes it accessible in the local browser.

  CLUSTER_NAME argument is the cluster name to execute the notebook on. It
  must already be present in the config file.

Options:
  -e, --environment TEXT  Environment path. Default: ~/.idact.conf or the
                          value of IDACT_CONFIG_PATH.
  --save-defaults         Save allocation parameters as defaults for next
                          time.
  --reset-defaults        Reset unspecified allocation parameters to defaults.
  --nodes INTEGER         Cluster node count. [Allocation parameter]. Jupyter
                          notebook will be deployed on the first node.
                          Default: 1.
  --cores INTEGER         CPU core count per node. [Allocation parameter].
                          Default: 1
  --memory-per-node TEXT  Memory per node. [Allocation parameter]. Default

For example, to deploy a notebook on a cluster with the same parameters as above, you could call:

```
python -m idact.notebook pro --save-defaults --environment notebooks/.idact-env --nodes 2 --cores 2 --memory-per-node 10GiB --walltime 0:20:00 --native-arg --partition plgrid-testing --native-arg --account intdata
```

The flag `--save-defaults` is optional, but it saves the allocation parameters: next time, the following will have the same effect:
```
python -m idact.notebook pro --environment notebooks/.idact-env
```
The `--environment` argument is optional if you use the default environment location.

The allocation and notebook the application deploys can be pulled from the cluster.

## Deploy Dask

One-time config step (cluster-specific):

In [42]:
cluster.config.setup_actions.dask = ['module load plgrid/tools/python-intel/3.6.2']
cluster.config.scratch = '$SCRATCH'
save_environment('.idact-env')

In [43]:
dd = deploy_dask(nodes)
dd

2018-10-22 02:33:48 INFO: Deploying Dask on 2 nodes.
2018-10-22 02:33:48 INFO: Connecting to p0118:35295 (1/2).
2018-10-22 02:33:48 INFO: Connecting to p0235:52865 (2/2).
2018-10-22 02:33:48 INFO: Deploying scheduler on the first node: p0118.
2018-10-22 02:33:58 WARNING: Exception: Unable to determine scheduler address., retry 1/5.


2018-10-22 02:34:05,824| ERROR   | Could not establish connection from ('127.0.0.1', 51678) to remote side of the tunnel
2018-10-22 02:34:05,826| ERROR   | Exception: Error reading SSH protocol banner
2018-10-22 02:34:05,829| ERROR   | Traceback (most recent call last):
2018-10-22 02:34:05,829| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\transport.py", line 2044, in _check_banner
2018-10-22 02:34:05,830| ERROR   |     buf = self.packetizer.readline(timeout)
2018-10-22 02:34:05,830| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\packet.py", line 353, in readline
2018-10-22 02:34:05,831| ERROR   |     buf += self._read_timeout(timeout)
2018-10-22 02:34:05,831| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\packet.py", line 542, in _read_timeout
2018-10-22 02:34:05,831| ERROR   |     raise EOFError()
2018-10-22 02:34:05,832| ERROR   | EOFError
2018-10-22 02:34:05,832| ERROR   | 
2018-10-22 02:34:05,832| ERROR

2018-10-22 02:34:05 WARNING: Exception: Could not establish session to SSH gateway, retry 1/3.


2018-10-22 02:34:12,645| ERROR   | Could not establish connection from ('127.0.0.1', 51689) to remote side of the tunnel
2018-10-22 02:34:12,647| ERROR   | Exception: Error reading SSH protocol banner
2018-10-22 02:34:12,649| ERROR   | Traceback (most recent call last):
2018-10-22 02:34:12,650| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\transport.py", line 2044, in _check_banner
2018-10-22 02:34:12,650| ERROR   |     buf = self.packetizer.readline(timeout)
2018-10-22 02:34:12,651| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\packet.py", line 353, in readline
2018-10-22 02:34:12,651| ERROR   |     buf += self._read_timeout(timeout)
2018-10-22 02:34:12,652| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\packet.py", line 542, in _read_timeout
2018-10-22 02:34:12,652| ERROR   |     raise EOFError()
2018-10-22 02:34:12,652| ERROR   | EOFError
2018-10-22 02:34:12,653| ERROR   | 
2018-10-22 02:34:12,653| ERROR

2018-10-22 02:34:12 WARNING: Exception: Could not establish session to SSH gateway, retry 1/3.
2018-10-22 02:34:18 INFO: Checking scheduler connectivity from p0118 (1/2).
2018-10-22 02:34:18 INFO: Checking scheduler connectivity from p0235 (2/2).
2018-10-22 02:34:20 INFO: Deploying workers.
2018-10-22 02:34:20 INFO: Deploying worker 1/2.


2018-10-22 02:35:04,520| ERROR   | Could not open connection to gateway


2018-10-22 02:35:04 WARNING: Exception: Could not establish session to SSH gateway, retry 1/3.


2018-10-22 02:35:19,743| ERROR   | Could not establish connection from ('127.0.0.1', 51701) to remote side of the tunnel
2018-10-22 02:35:19,745| ERROR   | Exception: Error reading SSH protocol banner
2018-10-22 02:35:19,748| ERROR   | Traceback (most recent call last):
2018-10-22 02:35:19,748| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\transport.py", line 2044, in _check_banner
2018-10-22 02:35:19,749| ERROR   |     buf = self.packetizer.readline(timeout)
2018-10-22 02:35:19,749| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\packet.py", line 353, in readline
2018-10-22 02:35:19,749| ERROR   |     buf += self._read_timeout(timeout)
2018-10-22 02:35:19,750| ERROR   |   File "E:\Anaconda3\envs\idact-dev\lib\site-packages\paramiko\packet.py", line 542, in _read_timeout
2018-10-22 02:35:19,750| ERROR   |     raise EOFError()
2018-10-22 02:35:19,751| ERROR   | EOFError
2018-10-22 02:35:19,751| ERROR   | 
2018-10-22 02:35:19,751| ERROR

2018-10-22 02:35:19 WARNING: Exception: Could not establish session to SSH gateway, retry 1/3.
2018-10-22 02:35:25 INFO: Deploying worker 2/2.
2018-10-22 02:35:45 INFO: Validating worker 1/2.
2018-10-22 02:35:45 INFO: Validating worker 2/2.


DaskDeployment(scheduler=tcp://localhost:51687/tcp://172.20.64.118:59560, workers=2)

In [44]:
nodes[0].resources.memory_usage

GiB(0.3794097900390625)

Get Dask client:

In [45]:
client = dd.get_client()
client

Client Scheduler: tcp://localhost:51687 Dashboard: http://localhost:54189/status,Cluster Workers: 2 Cores: 4 Memory: 21.47 GB


In [46]:
nodes[0].resources.cpu_usage

5.0

Computation will work only if Python and library versions match:

In [47]:
#x = client.submit(lambda: value + 1, 10)
#x.result() == 11

Diagnostics servers are tunnelled:

In [48]:
dd.diagnostics.addresses

['http://localhost:54189', 'http://localhost:51710', 'http://localhost:51716']

To open diagnostics servers in new tabs:

In [49]:
dd.diagnostics.open_all()

In [50]:
dd.cancel()

2018-10-22 02:36:45 INFO: Cancelling worker deployment on p0235.
2018-10-22 02:36:52 INFO: Cancelling worker deployment on p0118.
2018-10-22 02:36:59 INFO: Cancelling scheduler deployment on p0118.


## Close

In [51]:
nodes.running()

True

In [52]:
nodes.cancel()

2018-10-22 02:37:15 INFO: Cancelling job 13791710.


In [53]:
nodes.running()

False

In [54]:
node.run('squeue')

'JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)'

## Push and pull the environment

When working on a cluster, it may be useful to synchronize idact config with the local machine. Pushing the environment will merge the local environment into the remote environment.

In [55]:
push_environment(cluster, path='~/.idact-notebook-env')

2018-10-22 02:37:24 INFO: Pushing the environment to cluster.
2018-10-22 02:37:26 ERROR: Failure: Getting file from node pro.cyfronet.pl: /net/people/plggarstka/.idact-notebook-env
2018-10-22 02:37:26 ERROR: Failure: Deserializing the environment from cluster.
2018-10-22 02:37:26 INFO: Remote environment is missing, current environment will be copied to cluster.


In [56]:
print(node.run('cat ~/.idact-notebook-env'))

{
    "clusters": {
        "pro": {
            "auth": "PUBLIC_KEY",
            "disableSshd": false,
            "host": "pro.cyfronet.pl",
            "installKey": true,
            "key": null,
            "notebookDefaults": {},
            "port": 22,
            "portInfoRetries": 5,
            "scratch": "$SCRATCH",
            "setupActions": {
                "dask": [
                    "module load plgrid/tools/python-intel/3.6.2"
                ],
                "jupyter": [
                    "module load plgrid/tools/python-intel/3.6.2"
                ]
            },
            "user": "plggarstka"
        }
    },
    "logLevel": 20
}


The reverse operation is pulling the environment, which merges the remote environment into the local environment. Machine-specific information like the private key path is skipped when pushing or pulling.

In [57]:
pull_environment(cluster, path='~/.idact-notebook-env')

2018-10-22 02:37:39 INFO: Pulling the environment from cluster.


The 'path' parameter is optional. It defaults to ~/.idact.conf, or the value of the remote IDACT_CONFIG_PATH environment variable.

In [58]:
node.run('rm -v ~/.idact-notebook-env')

'removed ‘/net/people/plggarstka/.idact-notebook-env’'

## Remove cluster

A cluster can be removed from the environment.

In [59]:
add_cluster(name='fake',
            user='fakeuser',
            host='fakehost',
            port=2222)

2018-10-22 02:37:52 INFO: No auth method specified, defaulting to password-based.


Cluster(fakehost, 2222, fakeuser, auth=AuthMethod.ASK, key=None, install_key=True, disable_sshd=False)

In [60]:
show_clusters()

{'pro': Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='E:\\shared\\uni\\eng-project\\notebooks\\../.notebook-ssh\\id_rsa_sy', install_key=False, disable_sshd=False),
 'fake': Cluster(fakehost, 2222, fakeuser, auth=AuthMethod.ASK, key=None, install_key=True, disable_sshd=False)}

In [61]:
remove_cluster('fake')

In [62]:
show_clusters()

{'pro': Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='E:\\shared\\uni\\eng-project\\notebooks\\../.notebook-ssh\\id_rsa_sy', install_key=False, disable_sshd=False)}